<a href="https://colab.research.google.com/github/DryBoss/diarrhea-prediction-model/blob/main/diarrhea_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#data import

In [3]:
import pandas as pd

In [5]:
chattogram_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Chattogram.csv")
dhaka_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Dhaka.csv")
khulna_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Khulna.csv")
rajshahi_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Rajshahi.csv")